In [1]:
%load_ext autoreload
%autoreload 2
import os
os.chdir("/Users/Aaron-MAC/Development/Kaggle/Santander")
import pandas as pd
import numpy as np
from modules.pre_process import *
from modules.get_data import *
import matplotlib.pyplot as plt
import cPickle as pickle
import xgboost as xgb
%matplotlib inline

In [63]:
cache = pd.read_csv("data/train_ver2.csv").fillna(-2.)

In [65]:
target = get_target(get_training_data(year=2015, month=6, cache=cache))
target.head()

,ncodpers,diff_ind_ahor_fin_ult1,diff_ind_aval_fin_ult1,diff_ind_cco_fin_ult1,diff_ind_cder_fin_ult1,diff_ind_cno_fin_ult1,diff_ind_ctju_fin_ult1,diff_ind_ctma_fin_ult1,diff_ind_ctop_fin_ult1,diff_ind_ctpp_fin_ult1,...,diff_ind_hip_fin_ult1,diff_ind_plan_fin_ult1,diff_ind_pres_fin_ult1,diff_ind_reca_fin_ult1,diff_ind_tjcr_fin_ult1,diff_ind_valo_fin_ult1,diff_ind_viv_fin_ult1,diff_ind_nomina_ult1,diff_ind_nom_pens_ult1,diff_ind_recibo_ult1
0,16132,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1063040,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1063041,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1063042,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1063043,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
target.to_csv('train_target.csv', index=False)

In [44]:
cache = cache.loc[(cache['fecha_dato']>'2016-00-00') | (cache['fecha_dato']<'2015-07-00')]
cache_col_backup = cache.columns

new_columns = []
for item in cache.columns:
    if item.endswith("ult1"):
        new_columns.append("f_"+item)
    else:
        new_columns.append(item)
cache.columns = new_columns
cache.head()



,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,f_ind_hip_fin_ult1,f_ind_plan_fin_ult1,f_ind_pres_fin_ult1,f_ind_reca_fin_ult1,f_ind_tjcr_fin_ult1,f_ind_valo_fin_ult1,f_ind_viv_fin_ult1,f_ind_nomina_ult1,f_ind_nom_pens_ult1,f_ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [47]:
featured_cache = get_feature(cache)

In [56]:
test_set = get_test_data(year=2016, month=6, cache=cache)
test_set = get_feature(test_set)
test_set.head()

,fecha_dato,ncodpers,f_ind_ahor_fin_ult1,f_ind_aval_fin_ult1,f_ind_cco_fin_ult1,f_ind_cder_fin_ult1,f_ind_cno_fin_ult1,f_ind_ctju_fin_ult1,f_ind_ctma_fin_ult1,f_ind_ctop_fin_ult1,...,f_indresi,f_indext,f_canal_entrada,f_indfall,f_nomprov,f_ind_actividad_cliente,f_renta,f_segmento_01 - TOP,f_segmento_02 - PARTICULARES,f_segmento_03 - UNIVERSITARIO
0,2016-06-28,15889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,24,1.0,1.0,1,5.513391,1.0,0.0,0.0
1,2016-06-28,1170544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,24,1.0,31.0,0,5.513391,0.0,1.0,0.0
2,2016-06-28,1170545,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,150,1.0,14.0,1,5.513391,0.0,0.0,1.0
3,2016-06-28,1170547,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,150,1.0,2.0,0,5.171457,0.0,0.0,1.0
4,2016-06-28,1170548,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,150,1.0,3.0,0,5.028940,0.0,0.0,1.0


In [59]:
test_set.columns==featured_cache.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True], dtype=bool)

In [60]:
test_set.drop(['f_'+x for x in products], axis=1, inplace=True)
train_set = test_set
del test_set
train_set.head()

,fecha_dato,ncodpers,f_ind_empleado_A,f_ind_empleado_B,f_ind_empleado_F,f_ind_empleado_N,f_ind_empleado_P,f_pais_residencia,f_sexo,f_age,...,f_indresi,f_indext,f_canal_entrada,f_indfall,f_nomprov,f_ind_actividad_cliente,f_renta,f_segmento_01 - TOP,f_segmento_02 - PARTICULARES,f_segmento_03 - UNIVERSITARIO
0,2016-06-28,15889,0.0,0.0,1.0,0.0,0.0,79,1.0,11.2,...,0.0,1.0,24,1.0,1.0,1,5.513391,1.0,0.0,0.0
1,2016-06-28,1170544,0.0,0.0,0.0,1.0,0.0,79,0.0,7.2,...,0.0,1.0,24,1.0,31.0,0,5.513391,0.0,1.0,0.0
2,2016-06-28,1170545,0.0,0.0,0.0,1.0,0.0,79,1.0,4.4,...,0.0,1.0,150,1.0,14.0,1,5.513391,0.0,0.0,1.0
3,2016-06-28,1170547,0.0,0.0,0.0,1.0,0.0,79,0.0,4.4,...,0.0,1.0,150,1.0,2.0,0,5.171457,0.0,0.0,1.0
4,2016-06-28,1170548,0.0,0.0,0.0,1.0,0.0,79,0.0,4.4,...,0.0,1.0,150,1.0,3.0,0,5.028940,0.0,0.0,1.0


In [50]:
train_set = featured_cache.loc[featured_cache['fecha_dato']=='2015-06-28']
train_set.drop(['f_'+x for x in products], axis=1, inplace=True)
train_set.head()

/Users/Aaron-MAC/anaconda/envs/mlp/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,fecha_dato,ncodpers,f_ind_empleado_A,f_ind_empleado_B,f_ind_empleado_F,f_ind_empleado_N,f_ind_empleado_P,f_pais_residencia,f_sexo,f_age,...,f_indresi,f_indext,f_canal_entrada,f_indfall,f_nomprov,f_ind_actividad_cliente,f_renta,f_segmento_01 - TOP,f_segmento_02 - PARTICULARES,f_segmento_03 - UNIVERSITARIO
3144384,2015-06-28,16132,0.0,0.0,0.0,1.0,0.0,79.0,1.0,9.6,...,0.0,1.0,24.0,1.0,1.0,0.0,5.206572,0.0,1.0,0.0
3144385,2015-06-28,1063040,0.0,0.0,0.0,1.0,0.0,79.0,1.0,5.0,...,0.0,1.0,150.0,1.0,33.0,1.0,4.873313,0.0,0.0,1.0
3144386,2015-06-28,1063041,0.0,0.0,0.0,1.0,0.0,79.0,0.0,4.8,...,0.0,1.0,150.0,1.0,45.0,1.0,4.544797,0.0,0.0,1.0
3144387,2015-06-28,1063042,0.0,0.0,0.0,1.0,0.0,79.0,0.0,5.2,...,0.0,1.0,150.0,1.0,23.0,0.0,4.859688,0.0,0.0,1.0
3144388,2015-06-28,1063043,0.0,0.0,0.0,1.0,0.0,79.0,1.0,4.6,...,0.0,1.0,150.0,1.0,13.0,1.0,0.477121,0.0,0.0,1.0


In [61]:
train_set = pd.merge(train_set, featured_cache.loc[featured_cache['fecha_dato']=='2016-05-28', 'ncodpers':], how='left', on='ncodpers', suffixes=['', '_lag1'])
train_set = pd.merge(train_set, featured_cache.loc[featured_cache['fecha_dato']=='2016-04-28', 'ncodpers':], how='left', on='ncodpers', suffixes=['', '_lag2'])

train_set = pd.merge(train_set, featured_cache.loc[featured_cache['fecha_dato']=='2016-03-28', 'ncodpers':], how='left', on='ncodpers', suffixes=['', '_lag3'])
train_set = pd.merge(train_set, featured_cache.loc[featured_cache['fecha_dato']=='2016-02-28', 'ncodpers':], how='left', on='ncodpers', suffixes=['', '_lag4'])
train_set = pd.merge(train_set, featured_cache.loc[featured_cache['fecha_dato']=='2016-01-28', 'ncodpers':], how='left', on='ncodpers', suffixes=['', '_lag5'])
train_set.head()

,fecha_dato,ncodpers,f_ind_empleado_A,f_ind_empleado_B,f_ind_empleado_F,f_ind_empleado_N,f_ind_empleado_P,f_pais_residencia,f_sexo,f_age,...,f_indresi_lag5,f_indext_lag5,f_canal_entrada_lag5,f_indfall_lag5,f_nomprov_lag5,f_ind_actividad_cliente_lag5,f_renta_lag5,f_segmento_01 - TOP_lag5,f_segmento_02 - PARTICULARES_lag5,f_segmento_03 - UNIVERSITARIO_lag5
0,2016-06-28,15889,0.0,0.0,1.0,0.0,0.0,79,1.0,11.2,...,0.0,1.0,24.0,1.0,1.0,1.0,5.513391,1.0,0.0,0.0
1,2016-06-28,1170544,0.0,0.0,0.0,1.0,0.0,79,0.0,7.2,...,0.0,1.0,24.0,1.0,31.0,0.0,0.477121,0.0,1.0,0.0
2,2016-06-28,1170545,0.0,0.0,0.0,1.0,0.0,79,1.0,4.4,...,0.0,1.0,150.0,1.0,14.0,1.0,0.477121,0.0,0.0,1.0
3,2016-06-28,1170547,0.0,0.0,0.0,1.0,0.0,79,0.0,4.4,...,0.0,1.0,150.0,1.0,2.0,0.0,5.171457,0.0,0.0,1.0
4,2016-06-28,1170548,0.0,0.0,0.0,1.0,0.0,79,0.0,4.4,...,0.0,1.0,150.0,1.0,3.0,0.0,5.028940,0.0,0.0,1.0


In [62]:
train_set.to_csv("test_feature.csv", index=False)

In [16]:
X_train, y_train = feed_training_data(year=2015, month=6, cache=cache, mul_y=False)

X_test = feed_test_data(year=2016, month=6, cache=cache)
X_test_unred = X_test.copy()
test_set = get_test_data(year=2016, month=6, cache=cache)

In [33]:
sel = np.in1d(y_train, [2,4,6,12,17,18,21,22,23])
X_train = X_train[sel]
y_train = y_train[sel]

y_train[y_train==22] = 21

In [18]:
arr = []
for i in range(24):
    sel = y_train==i
    
    if len(X_train[sel])>0:
        arr.append(X_train[sel].mean(0))
feature_sel = np.array(arr).std(0)

X_train = X_train[:, feature_sel>0.01]
X_test = X_test[:, feature_sel>0.01]

In [34]:
X_train.shape

(43189, 35)

In [36]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(verbose=True, n_estimators=50)
model.classes_ = range(0, 24)

model = model.fit(X_train, y_train)
predict = model.predict_proba(X_test)
predict.shape
#predict = np.hstack([np.zeros((len(predict),2)), predict])

      Iter       Train Loss   Remaining Time 
         1       76812.0861           30.52s
         2       70476.1239           30.97s
         3       65757.8052           30.49s
         4       62103.8840           28.96s
         5       59138.2666           28.27s
         6       56737.3472           27.61s
         7       54659.0130           26.85s
         8       52942.5494           26.20s
         9       51509.5934           25.94s
        10       50291.5534           25.29s
        20       44008.9849           18.39s
        30       41883.6951           12.13s
        40       40833.1724            5.99s
        50       40211.1029            0.00s


(929615, 8)

In [37]:
model.classes_

array([ 2,  4,  6, 12, 17, 18, 21, 23])

In [21]:
mask_1 = np.ones((len(predict), 24))
mask_1[:, [0,1,3,5,7,8,9,10,11,13,14,15,16,19,20]] = 0.01
mask_2 = X_test_unred[:, :24]==0

predict = predict*mask_1*mask_2

In [22]:
f = open("outputs/dec_12_sub.csv", 'w')
f.write("ncodpers,added_products\n")

user_id = test_set['ncodpers']
print len(predict)

for idx,r in enumerate(predict):
    order = [i[0] for i in sorted(enumerate(r), key=lambda x:x[1], reverse=True)]
    select = order[:min(7,len(order))]
    f.write(str(user_id[idx])+","+(" ".join([products[s] for s in select]))+'\n')
f.close()

929615


In [8]:
prod = cache.values[:, -24:]

In [10]:
from sklearn.neural_network import BernoulliRBM

rbm = BernoulliRBM(n_components=10, verbose=True, learning_rate=1e-3, n_iter=10)
rbm = rbm.fit(prod)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -3.03, time = 113.81s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -2.99, time = 145.17s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -2.98, time = 142.50s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -2.93, time = 141.27s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -2.95, time = 138.78s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -2.90, time = 146.38s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -2.94, time = 145.52s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -2.88, time = 142.06s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -2.93, time = 139.33s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -2.91, time = 141.15s
